In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
file_path = "file:///home/talentum/shared/chandra_cleaned.csv"


# Create an DataFrame from file_path
heart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
heart_df.createOrReplaceTempView("new_heart")

In [5]:
query=spark.sql("select X_MICHD from new_heart")

In [6]:
query.take(2)

[Row(X_MICHD=0.0), Row(X_MICHD=0.0)]

## Analysis On Features

### Analyzing Age column

In [45]:
query=spark.sql("select (case when X_AGE_G =1 then 'AGE_18_24' when X_AGE_G=2 then 'AGE_25_34' when X_AGE_G=3 then 'AGE_35_44' when X_AGE_G=4 then 'AGE_45_54' when X_AGE_G=5 then 'AGE_55_64' when X_AGE_G=6 then 'AGE_Above_64' end) as AGE_CATEGORY, count(X_AGE_G)as cnt from new_heart where X_MICHD=1.0 group by X_AGE_G order by cnt desc") 

In [47]:
query.collect()

[Row(AGE_CATEGORY='AGE_Above_64', cnt=22824),
 Row(AGE_CATEGORY='AGE_55_64', cnt=6734),
 Row(AGE_CATEGORY='AGE_45_54', cnt=2912),
 Row(AGE_CATEGORY='AGE_35_44', cnt=1036),
 Row(AGE_CATEGORY='AGE_25_34', cnt=477),
 Row(AGE_CATEGORY='AGE_18_24', cnt=180)]

In [ ]:
# Among Detected Heart Disease, count of people above age of 64 is more

### Analyzing Body Mass Index 

In [35]:
query1=spark.sql("select (case when X_BMI5CAT=1.0 then 'Underweight' when X_BMI5CAT=2.0 then 'Normal Weight' when X_BMI5CAT=3.0 then 'Overweight' when X_BMI5CAT=4.0 then 'Obese' end) as BMI5CAT, count(X_BMI5CAT)as cnt from new_heart where X_MICHD=1.0 group by X_BMI5CAT order by cnt desc")

In [36]:
query1.collect()

[Row(BMI5CAT='Overweight', cnt=13975),
 Row(BMI5CAT='Obese', cnt=12254),
 Row(BMI5CAT='Normal Weight', cnt=7432),
 Row(BMI5CAT='Underweight', cnt=502)]

In [ ]:
# Among Detected Heart disease, count of Overweight category is more compared with other categories


### Analyzing Gender 

In [48]:
query2=spark.sql("select (case when SEXVAR=1 then 'Male' when SEXVAR=2 then 'Female' end) as Gender, count(SEXVAR) cnt_gen from new_heart where X_MICHD=1.0 group by SEXVAR order by cnt_gen desc")

In [50]:
query2.collect()

[Row(Gender='Male', cnt_gen=19477), Row(Gender='Female', cnt_gen=14686)]

In [51]:
# Among Detected heart Disease, count of Males are more

### Analyzing Marital

In [57]:
query3=spark.sql("select (case when MARITAL=1.0 then 'Married' when MARITAL=2.0 then 'Divorced' when MARITAL=3.0 then 'Widowed' when MARITAL=4.0 then 'Seperated' when MARITAL=5.0 then 'Never_Maried' when MARITAL=6.0 then 'Mem' end )as Marital, count(MARITAL)as cnt from new_heart where X_MICHD=1.0 group by MARITAL order by cnt desc")

In [58]:
query3.collect()

[Row(Marital='Married', cnt=16593),
 Row(Marital='Widowed', cnt=7600),
 Row(Marital='Divorced', cnt=5824),
 Row(Marital='Never_Maried', cnt=2780),
 Row(Marital='Seperated', cnt=743),
 Row(Marital='Mem', cnt=623)]

In [59]:
# Among Detected Heart Disease , count of Married people is more

### Analyzing Smoker

In [61]:
query4=spark.sql("select (case when X_SMOKER3=1 then 'Everyday_smoker' when X_SMOKER3=2 then 'Smokes_someday' when X_SMOKER3=3 then 'Former_Smoker' when X_SMOKER3=4 then 'Never_Smoked' end) as Smoke, count(X_SMOKER3)as cnt from new_heart where X_MICHD=1.0 group by X_SMOKER3 order by cnt desc")

In [62]:
query4.collect()

[Row(Smoke='Never_Smoked', cnt=15113),
 Row(Smoke='Former_Smoker', cnt=13650),
 Row(Smoke='Everyday_smoker', cnt=4040),
 Row(Smoke='Smokes_someday', cnt=1360)]

In [ ]:
# Among Detected Heart Disease, count of Never Smoked is more

### Analyzing General Health

In [64]:
query5=spark.sql("select (case when GENHLTH=1 then 'Excellent' when GENHLTH=2 then 'Very Good' when GENHLTH=3 then 'Good' when GENHLTH=4 then 'Fair' when GENHLTH=5 then 'Poor' end)as General_Health , count(GENHLTH)as cnt from new_heart where X_MICHD=1.0 group by GENHLTH order by cnt desc")

In [65]:
query5.collect()

[Row(General_Health='Good', cnt=11657),
 Row(General_Health='Fair', cnt=9128),
 Row(General_Health='Very Good', cnt=6522),
 Row(General_Health='Poor', cnt=5029),
 Row(General_Health='Excellent', cnt=1827)]

In [ ]:
# Among Detected with Heart Disease, count of Good Genral Health People is more 

### Analyzing Mental Health

In [70]:
query6=spark.sql("select (case when X_MENT14D=1 then 'Good_Mental_Health' when X_MENT14D=2 then 'Short_Term_Mental_Health' when X_MENT14D=3 then 'Long_Term_Mental_Health' end )as mental_health , count(X_MENT14D)as Ment from new_heart where x_MICHD=1.0 group by X_MENT14D order by Ment desc")

In [71]:
query6.collect()

[Row(mental_health='Good_Mental_Health', Ment=23036),
 Row(mental_health='Short_Term_Mental_Health', Ment=5886),
 Row(mental_health='Long_Term_Mental_Health', Ment=5241)]

In [ ]:
# Among Detected with Heart Disease , count of Good Mental Health is more compared to Long Term Mental Health

### Analyzing Asthma 

In [75]:
query8=spark.sql("select (case when ASTHMA3=1 then 'YES' when ASTHMA3=2 then 'No' end)as Asthma , count(ASTHMA3) as cnt from new_heart where X_MICHD=1.0 group by ASTHMA3 order by cnt desc")

In [76]:
query8.collect()

[Row(Asthma='No', cnt=27839), Row(Asthma='YES', cnt=6324)]

In [ ]:
# Among Detected Heart Disease count of people having Asthma is less compared to not detected with Asthma

### Analyzing Stroke

In [78]:
query9=spark.sql("select (case when CVDSTRK3=1 then 'Yes' when CVDSTRK3=2 then 'No' end) as Stroke, count(CVDSTRK3) as cnt from new_heart where X_MICHd=1.0 group by CVDSTRK3 order by cnt")

In [79]:
query9.collect()

[Row(Stroke='Yes', cnt=5584), Row(Stroke='No', cnt=28579)]

In [ ]:
# Among Detected Heart Disease count of people not having stroke is more

### analyzing RACE

In [84]:
query10=spark.sql("select (case when X_IMPRACE=1 then 'White' when X_IMPRACE=2 then 'Black' when X_IMPRACE=3 then 'Asian' when X_IMPRACE=4 then 'Non-Hispanic' when X_IMPRACE=5 then 'Hispanic' end) as Race ,count(X_IMPRACE)as cnt from new_heart where X_MICHD=1.0 group by X_IMPRACE order by cnt desc ")

In [85]:
query10.collect()

[Row(Race='White', cnt=27701),
 Row(Race='Black', cnt=2329),
 Row(Race='Hispanic', cnt=1901),
 Row(Race=None, cnt=1186),
 Row(Race='Non-Hispanic', cnt=715),
 Row(Race='Asian', cnt=331)]

In [86]:
# Among Detected heart disease white people are having higher count

### Analyzing Kidney Disease

In [87]:
query11=spark.sql("select (case when CHCKDNY2=1 then 'Yes' when CHCKDNY2=2 then 'No' end )as Kidney , count(CHCKDNY2)as cnt from new_heart where X_MICHD=1.0 group by CHCKDNY2 order by cnt desc")

In [88]:
query11.collect()

[Row(Kidney='No', cnt=29810), Row(Kidney='Yes', cnt=4353)]

In [ ]:
# Among Detected Heart Disease count of People not having kidney disease is more 

### Analyzing Skin cancer

In [89]:
query12=spark.sql("select (case when CHCSCNCR=1 then 'Yes' when CHCSCNCR=2 then 'No' end)as Skin_Cancer , count(CHCSCNCR)as cnt from new_heart where X_MICHD=1.0 group by CHCSCNCR order by cnt")

In [90]:
query12.collect()

[Row(Skin_Cancer='Yes', cnt=6097), Row(Skin_Cancer='No', cnt=28066)]

In [91]:
# Among Detected Heart Disease count of people not having Skin Cancer is more

In [92]:
query12=spark.sql("select (case when CHCOCNCR=1 then 'Yes' when CHCOCNCR=2 then 'No' end)as Cancer , count(CHCOCNCR)as cnt from new_heart where X_MICHD=1.0 group by CHCOCNCR order by cnt")

In [93]:
query12.collect()

[Row(Cancer='Yes', cnt=6119), Row(Cancer='No', cnt=28044)]

In [94]:
# among Detected Heart Disease count of people not having any type of cancer is more

### Anlayzing Depressive Disorder 

In [97]:
query13=spark.sql("select (case when  ADDEPEV3=1 then 'Yes' when  ADDEPEV3=2 then 'No' end) as Depressive_Disorder , count(ADDEPEV3) as cnt from new_heart group by  ADDEPEV3 order by cnt desc")

In [98]:
query13.collect()

[Row(Depressive_Disorder='No', cnt=326138),
 Row(Depressive_Disorder='Yes', cnt=75820)]

In [99]:
# Count of Non Depressive People is more